# 测试通过嵌入检索的成功率 - 生成中文摘要的情况

初步结论：

- k=1的时候，召回率达到72%
- 到5个的时候，召回率72%，没有变化
- 即使k=n-1, 召回率都不是100%，说明相似度方式在中文下相似度设置高了也不一定有用
- 中文的文档摘要，和英文版本比较，缺少了问题部分

后续改进，必须换其他检索方式，比如融合检索，k=1&&关键字&&bm25

## 准备

In [1]:
%%time

INDEX_PATH="retrieve-zh-index"
DATA_PATH="retrieve-zh-data"

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


In [2]:
%%time

test_data=[
    {
        "url": "https://www.guancha.cn/ZiZheng/2024_06_27_739401_s.shtml",
        "question": "和平共处五项原则为啥到现在还有重要意义？"
    },
    {
        "url": "https://www.guancha.cn/kegongliliang/2024_06_27_739408_s.shtml",
        "question": "丰鸟科技在内蒙古主场地的资产清点和撤出的具体原因是什么？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257033",
        "question": "电池工厂火灾的具体起因是什么？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257049",
        "question": "莫迪计划推出的亲商改革具体包括哪些措施？"
    },
    {
        "url": "https://www.guancha.cn/politics/2024_06_26_739354.shtml",
        "question": "金融监管总局支持信托公司和理财公司加大创业投资力度的具体措施有哪些？"
    },
    {
        "url": "https://www.guancha.cn/gaotianwei/2024_06_24_739062_s.shtml",
        "question": "我们为什么要发展可重复使用运载火箭的验证箭？"
    },
    {
        "url": "https://www.guancha.cn/economy/2024_06_26_739352.shtml",
        "question": "中小银行合并重组的原因是什么？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739492.shtml",
        "question": "除了菲律宾，日本政府还计划在哪些东南亚国家推广5G建设计划？"
    },
    {
        "url": "https://www.guancha.cn/politics/2024_06_27_739480.shtml",
        "question": "尤勇智对年轻演员有何建议？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257405",
        "question": "俄罗斯如何对待塞尔维亚总统武契奇的表态？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739443_s.shtml",
        "question": "梅洛尼在欧盟高层人选提名问题上的态度是什么？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739432.shtml",
        "question": "金砖国家为什么决定暂时停止接纳新成员国？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739441.shtml",
        "question": "印尼旅游业面临的竞争压力有哪些？"
    },
    {
        "url": "https://www.guancha.cn/XueKaiHuan/2024_06_24_739023_s.shtml",
        "question": "波兰近年来在对华政策上采取的“双轨政策”具体体现在哪些方面？"
    },
    {
        "url": "https://www.guancha.cn/economy/2024_06_27_739420.shtml",
        "question": "为了加强技能人才培养并解决结构性就业矛盾，国务院常务会议部署了哪些具体措施？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257296",
        "question": "以色列内塔尼亚胡所述的“激战阶段”如何定义？"
    },
    {
        "url": "https://www.guancha.cn/ShiYang/2024_06_23_738937_s.shtml",
        "question": "菲律宾为何选择派遣运补船和充气艇非法进入中国南沙群岛仁爱礁邻近海域？"
    },
    {
        "url": "https://www.guancha.cn/fengchao/2024_06_23_738933_s.shtml",
        "question": "普京访越期间，俄罗斯和越南就哪些领域的合作达成了新的协议或共识？"
    }
]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [3]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data

CPU times: user 96.3 ms, sys: 87.1 ms, total: 183 ms
Wall time: 83.6 ms


In [4]:
%%time

!mkdir -p $DATA_PATH

import json

for news in test_data:
    data=get_news_data(news['url'])
    data['url']=news['url']
    file_path = f'./{DATA_PATH}/{data["title"]}.json'
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

CPU times: user 2 s, sys: 7.34 ms, total: 2 s
Wall time: 4.52 s


In [5]:
%%time

from llama_index.core import(
    Document
)

def data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2.45 s, sys: 313 ms, total: 2.76 s
Wall time: 2.57 s


In [6]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir=f"./{DATA_PATH}").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

import json

docs=[]
for document in documents:
    news_data=json.loads(document.text) # documents[0].text
    docs.append(data2doc(news_data))

documents=docs

len(docs)

CPU times: user 25.5 ms, sys: 554 µs, total: 26.1 ms
Wall time: 3.1 s


18

In [7]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 325 µs, sys: 845 µs, total: 1.17 ms
Wall time: 854 µs


In [8]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 682 ms, sys: 27.5 ms, total: 710 ms
Wall time: 710 ms


In [9]:
%%time

from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# QA系统提示
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
        "您是全世界值得信赖的 QA 系统。\n"
        "始终使用提供的上下文信息而不是先验知识来回答查询。\n"
        "需要遵循的一些规则：\n"
        "1. 不要在答案中直接引用指定的上下文。\n"
        "2. 避免诸'基于上下文...'或'上下文信息...'或类似的语句。 "
    ),
    role=MessageRole.SYSTEM,
)

# QA 提示模板消息
TEXT_QA_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "上下文信息如下。\n"
            "----------\n"
            "{context_str}\n"
            "----------\n"
            "给定上下文信息而非先验知识，"
            "查询：{query_str}\n"
            "回答： "
        ),
        role=MessageRole.USER,
    ),
]

# 聊天QA提示
CHAT_TEXT_QA_PROMPT = ChatPromptTemplate(message_templates=TEXT_QA_PROMPT_TMPL_MSGS)


# 树状摘要提示信息
TREE_SUMMARIZE_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "下面提供了来自多个来源的上下文信息。\n"
            "----------\n"
            "{context_str}\n"
            "----------\n"
            "鉴于来自多个来源的信息而非先验知识，"
            "回答查询。\n"
            "查询：{query_str}\n"
            "回答： "
        ),
        role=MessageRole.USER,
    ),
]

# 树状摘要提示
CHAT_TREE_SUMMARIZE_PROMPT = ChatPromptTemplate(
    message_templates=TREE_SUMMARIZE_PROMPT_TMPL_MSGS
)

# 摘要查询
SUMMARY_QUERY = "请总结所提供文本的内容。"

CPU times: user 0 ns, sys: 138 µs, total: 138 µs
Wall time: 140 µs


In [10]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", 
    use_async=True,
    text_qa_template=CHAT_TEXT_QA_PROMPT,
    summary_template=CHAT_TREE_SUMMARIZE_PROMPT,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
    summary_query=SUMMARY_QUERY,
)

Parsing nodes:   0%|          | 0/18 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/18 [00:00<?, ?it/s]

current doc id: “塞铁”跳反了？塞尔维亚“背刺”俄罗斯：承认已暗中向乌克兰提供8亿美元弹药！
current doc id: “对抗中国技术”，日本计划帮菲律宾建设5G通信网络
current doc id: 一周军评：菲律宾“后继乏力”，我们的考验又在哪里？
current doc id: 丁刚：中印能实现龙象共舞吗？
current doc id: 中东又一场战争可能要爆发了！
current doc id: 中小银行改革加速度，今年来超80家开启合并重组模式
current doc id: 冯超：普京访越，透视出的国家相处之道应该是什么？
current doc id: 印尼考虑对华免签，预计10月份前敲定
current doc id: 国常会：一视同仁支持内外资企业参与大规模设备更新、政府采购和投资
current doc id: 子政：和平共处五项原则历久弥新的原因是什么？
current doc id: 尤勇智：不可想象！现在素人、网红、有背景的人都能来演戏
current doc id: 拉夫罗夫：金砖十国已投票决定，暂时停止扩员以专注整合新成员国
current doc id: 欧盟“内定”冯德莱恩连任，意大利总理警告：注定在欧洲议会遇到困难
current doc id: 科工力量：前有高铁，后有快递小哥，支线物流在中国飞不起来？
current doc id: 薛凯恒：日渐亲美的波兰，是否会改变一贯的对华中立政策？
current doc id: 金融监管总局：将研究提高保险资金投资创业投资基金集中度的比例上限
current doc id: 韩国电池厂火灾17名中国人遇难：有多少同胞，在韩国打工？
current doc id: 高天伟：国产可重复火箭完成10公里测试，有一点与马斯克很不一样


Generating embeddings:   0%|          | 0/18 [00:00<?, ?it/s]

CPU times: user 1.16 s, sys: 55.8 ms, total: 1.21 s
Wall time: 8min 16s


In [11]:
doc_summary_index.get_document_summary("丁刚：中印能实现龙象共舞吗？")

'所提供的文本主要讨论了印度总理纳伦德拉·莫迪的第三个任期及其对印度制造业和中印关系的影响。文章首先提到莫迪计划在新任期内推出一系列亲商改革以提升印度制造业竞争力，将中国视为主要竞争对手。然而，文章指出印度发展制造业面临的主要挑战包括劳动力素质、教育水平、社会与传统价值观制约、种姓制度限制低种姓群体的教育和职业机会等。\n\n文章进一步分析了莫迪政府在经济领域的目标，即到2047年将印度在全球制造业中的份额提高至10%，但同时指出印度制造业占GDP比重从2016年的15.16%下降到了2021年的13.98%，而同期中国制造业的份额一直稳定在26%以上。文章强调了劳动力素质问题，如低参与度、性别差异（女性参与率远低于中国）以及土地和少数民族问题等。\n\n文章还讨论了印度制造业对中国的依赖性，特别是在设备、中间产品和原材料方面，即使在其主要出口产业纺织业也是如此。美国和西方试图重组全球供应链为印度制造业带来了机遇，但真正的挑战在于如何在重构供应链的背景下采取更为自主的对华经贸政策，并吸引投资。\n\n文章指出莫迪政府可能会实施更加自信的外交政策，特别是在针对中国安全风险和边境问题上可能表现得更为强硬。然而，从近两年的对华政策来看，求稳仍是主调，下一步应管控引发冲突的问题并继续推进与中国的经贸合作。文章强调了通过合作实现印度制造业发展的动力，并指出中印之间的合作规模和速度与双方的改革开放密切相关。\n\n最后，文章引用邓小平的话，提到中国、印度和其他邻国的发展对于亚洲乃至全球的重要性。文章认为只有自信且有能力迈出轻盈步伐的印度才能让中印“龙象共舞”成为可能，这需要双方对自身发展道路有更多的自信，并在双边交往中展开更宽广的合作。\n\n总结而言，文本探讨了莫迪政府在新任期内推动制造业发展的挑战、印度与中国的经济关系以及中印合作的重要性。'

In [12]:
%%time

doc_summary_index.storage_context.persist(INDEX_PATH)

CPU times: user 56.2 ms, sys: 3.99 ms, total: 60.2 ms
Wall time: 59.4 ms


In [13]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=INDEX_PATH)
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 106 ms, sys: 0 ns, total: 106 ms
Wall time: 105 ms


## 检索

### 简单测试

In [14]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve(test_data[0]["question"])
retrieved_nodes[0].metadata

CPU times: user 6.34 ms, sys: 0 ns, total: 6.34 ms
Wall time: 105 ms


{'title': '子政：和平共处五项原则历久弥新的原因是什么？',
 'publish_time': '2024-06-27 07:32:02',
 'author': '郭涵',
 'url': 'https://www.guancha.cn/ZiZheng/2024_06_27_739401_s.shtml',
 'images': ['https://i.guancha.cn/news/official/2024/06/26/20240626151444637.jpg',
  'https://i.guancha.cn/news/official/2024/06/26/20240626152937401.jpeg',
  'https://i.guancha.cn/news/official/2024/06/26/20240626153929709.png',
  'https://i.guancha.cn/shiping-banner.jpg']}

### k=1

In [15]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=1,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results

CPU times: user 78.5 ms, sys: 305 µs, total: 78.8 ms
Wall time: 1.68 s


[True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True]

In [16]:
results.count(True) / len(results)

0.7222222222222222

In [18]:
# 科工力量：前有高铁，后有快递小哥，支线物流在中国飞不起来？
# 丰鸟科技在内蒙古主场地的资产清点和撤出的具体原因是什么？
doc_summary_index.get_document_summary("科工力量：前有高铁，后有快递小哥，支线物流在中国飞不起来？")

'所提供的文本主要围绕中国支线物流的发展现状、面临的挑战及与美国等国家的对比展开。文章首先通过新疆库车和轮台等地的小白杏运输问题，强调了支线物流在时间敏感型高价值商品运输中的重要性，并指出西部地区交通网络相对稀疏导致农产品需要经过多级中转的问题。\n\n接着，文本分析了中国交通基础设施的历史发展与现状，特别是政府投资主要集中在铁路、公路等“铁公基”项目上。文章提到，在东部地区交通设施发展迅速的同时，西部地区的交通瓶颈限制了其经济发展潜力，并以贵州六枝特大桥为例，强调了西部地区基础设施建设的挑战。\n\n文本进一步讨论了中国支线物流的发展困境，指出与高铁和公路等干线运输相比，支线航空在成本和效率方面面临竞争压力。尽管支线物流有独特优势，但在全国范围内实现全面发展的挑战较大。\n\n文章还对比了中国和美国在通航（通用航空）行业的发展现状、挑战以及未来可能的解决方案。关键点包括人才短缺问题、高成本结构、电动化与智能化技术带来的可能性，以及顺丰等企业通过自主研发和投资无人机领域进行探索的努力。\n\n最后，文本总结了中国物流行业中的支线运输问题、顺丰在这一领域的尝试及成功案例分析，并提到了美国联邦快递与赛斯纳208“大篷车”飞机的成功合作。文章强调了中国在发展通航物流时面临的挑战以及通过技术创新寻求解决方案的可能性。\n\n综合来看，文本探讨了以下几点：\n1. **赛斯纳208系列飞机**：自1982年开始生产，至今仍在持续生产，并且累计交付数量超过3500架。年产量稳定在80至100架之间。\n   \n2. **顺丰面临的挑战与策略**：顺丰作为快递运营公司，在自主研发和制造飞机方面面临资金投入的挑战，尤其是在“脱钩断链”的背景下。顺丰尝试通过“垂直整合”模式直接研发制造飞机，但这一策略在需求端遇到了问题。\n\n3. **发展支线物流航空**：文章指出中国在发展支线物流航空领域具有优势，包括强大的实体经济、激活的下沉市场、成熟的基础设施以及时代变量（如电动化和智能化）的影响。同时，也指出了国内航空供应链和业态与国外相比存在差距，并提到了脱钩断链的风险。\n\n4. **联邦快递与赛斯纳的成功模式**：文章提到联邦快递与赛斯纳的合作是成功的案例，通过深度沟通和技术合作，双方共同研发了满足特定业务需求的飞机型号。这种模式被建议为中国顺丰等公司提供参考。\n\n总结而言，

In [19]:
# "欧盟“内定”冯德莱恩连任，意大利总理警告：注定在欧洲议会遇到困难"
# 梅洛尼在欧盟高层人选提名问题上的态度是什么？
doc_summary_index.get_document_summary("欧盟“内定”冯德莱恩连任，意大利总理警告：注定在欧洲议会遇到困难")

'所提供的文本主要关注了欧盟内部关于未来五年的领导层人选讨论和决策过程。关键点包括：\n\n1. **冯德莱恩连任**：意大利总理梅洛尼对这一决定表示批评，并警告称被提名者在欧洲立法机构中将遇到困难，因为她的党团“欧洲保守派和改革主义者”在最近的选举中获得83个席位，跃升为第三大党团。\n\n2. **提名过程**：6名欧盟领导人已经同意推举冯德莱恩担任欧盟委员会主席、安东尼奥·科斯塔担任欧洲理事会主席以及卡娅·卡拉斯出任欧盟外交与安全政策高级代表。这六位领导人的党团分别来自欧洲人民党（EPP）、“社会主义者和民主人士进步联盟”（S&D）和“复兴欧洲”（RE）。\n\n3. **政治动态**：梅洛尼认为，“欧洲保守派和改革主义者”理应拥有关键发言权，并批评欧盟领导人试图掩盖选民对欧盟的不满。匈牙利总理欧尔班也对提名过程表示不满，认为与左翼和自由派达成的协议违背了欧盟的基础原则。\n\n4. **朔尔茨的支持**：德国总理朔尔茨证实了欧洲议会三个主要中间派党团支持冯德莱恩、科斯塔和卡拉斯担任相关职位的消息，并希望在峰会上达成协议以避免僵局。\n\n5. **后续步骤**：预计欧盟领导人将在峰会上批准提名协议，但冯德莱恩面临的更大挑战是获得欧洲议会的支持。她需要至少361席的票数来确保连任，而历史上大约有10%的欧洲议员会在秘密投票中改变立场，这使得她的优势变得微弱。\n\n6. **潜在交易策略**：为了争取梅洛尼的支持，冯德莱恩可能考虑在新一届欧委会中为意大利安排一个更具影响力的职位。外交官们预计欧盟领导人在布鲁塞尔峰会上会批准提名协议，但更关键的问题在于她能否赢得欧洲议会的支持。\n\n整体而言，这篇报道强调了冯德莱恩连任过程中的政治挑战和潜在的交易策略，特别是在获得欧洲议会支持方面。'

### k=2

In [17]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=2,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 91.5 ms, sys: 698 µs, total: 92.2 ms
Wall time: 1.7 s


0.7222222222222222

### k=3

In [20]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 103 ms, sys: 1.24 ms, total: 104 ms
Wall time: 2.73 s


0.7222222222222222

### k=5

In [21]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=5,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 116 ms, sys: 4.92 ms, total: 121 ms
Wall time: 1.74 s


0.7222222222222222

### k=len(test_data)

In [24]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=len(test_data),
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 291 ms, sys: 3.59 ms, total: 295 ms
Wall time: 1.9 s


1.0

In [23]:
len(test_data)

18

In [24]:
doc_summary_index.get_document_summary("丁刚：中印能实现龙象共舞吗？")

'所提供的文本主要讨论了印度总理纳伦德拉·莫迪的第三个任期及其对印度制造业发展的雄心。文章指出，莫迪政府计划推出一系列亲商改革以提升印度制造业的竞争力，并将其视为与中国的竞争对象。然而，实现这一目标面临诸多挑战，包括劳动力素质、教育水平、社会传统和价值观制约等问题。\n\n文本强调了印度在经济领域依赖中国设备、中间产品和原材料的情况，特别是在纺织业等主要出口产业中。美国和西方试图重组全球供应链为印度制造业带来了机遇，但真正的挑战在于如何在美国的推动下采取更为自主的对华经贸政策，并吸引投资。\n\n文章还讨论了莫迪政府可能实施更加自信的外交政策，尤其是在针对中国安全风险和边境问题上可能会表现得更为强硬。然而，从近两年的对华政策来看，求稳仍是主调。文本认为，印度与中国的供应链和产业链连接对于双方都至关重要。\n\n最后，文章提到成功的工业化不仅需要正确的经济政策，还需要与当地劳动力的文化态度、职业精神和价值观相吻合。自印度独立以来，任何有助于提升国家工业化的改革都因触及传统而面临挑战。因此，莫迪政府是否能进行具有前瞻性的改革以及拥有高政治智慧和强大的舆论引导力将决定其能否实现制造业目标。\n\n文章总结指出，中印合作的规模和速度与双方的改革开放密切相关。如果两个大国都能自信地展开双边交往，并扩大合作领域，减少战略猜忌，那么“龙象共舞”将成为世界舞台上引人注目的场景。邓小平在1988年的讲话中也强调了中印合作的重要性，认为中国、印度和其他邻国的发展将标志着真正的亚洲世纪的到来。'